In [1]:
#import opustools_pkg

In [2]:
#opus_reader = opustools_pkg.OpusRead(["-d", "OpenSubtitles2018", "-s", "en", "-t", "ru"
#                                      ,"-rd", "/srv/hd5/data/vprov/Poems/data/"])
                                      #,"-wm", "moses", "-w", 
                                     #"/srv/hd5/data/vprov/Poems/data/c.clean.en,/srv/hd5/data/vprov/Poems/data/c.clean.ru"])

## DO NOT START HERE (BPE learning)

In [ ]:
from tqdm import tqdm_notebook
import re
import time

In [2]:
#en_list = []
#ru_list = []
#counter = 0
#start = time.time()
#regex_corner = re.compile('[<>?]')
#with open('/srv/hd6/data/Poem2Poem/data/ParallelEnRu/OpenSubtitlesv2018/en-ru.tmx', 'r') as f:
#    while True:
#        line = f.readline()
#        if not line:
#            break
#        if ":lang=\"en\"" in line:
#            good_tokens = regex_corner.sub(' ', line).split()[3:-2]
#            en_list.append(' '.join(good_tokens))
#            counter += 1
#        elif ":lang=\"ru\"" in line:
#            good_tokens = regex_corner.sub(' ', line).split()[3:-2]
#            ru_list.append(' '.join(good_tokens))
#            counter += 1
#        if counter % 100005 == 0:
#            print(counter, time.time()-start)
#            counter += 1

0 0.001195669174194336
100005 0.9053459167480469
200010 1.8064923286437988
300015 2.7129876613616943
400020 3.621779203414917
500025 4.536507606506348
600030 5.449934482574463
700035 6.354427814483643
800040 7.272990703582764
900045 8.191219568252563
1000050 9.093032121658325
1100055 9.999032497406006
1200060 10.907984018325806
1300065 11.814679861068726
1400070 12.711447954177856
1500075 13.613584041595459
1600080 14.511685132980347
1700085 15.410392999649048
1800090 16.301660776138306
1900095 17.19749903678894
2000100 18.099979162216187
2100105 18.993664026260376
2200110 19.889615535736084
2300115 20.777644634246826
2400120 21.669100999832153
2500125 22.565782070159912
2600130 23.463003396987915
2700135 24.356353998184204
2800140 25.258469820022583
2900145 26.148648023605347
3000150 27.06009006500244
3100155 27.965386867523193
3200160 28.860894203186035
3300165 29.76190757751465
3400170 30.657077074050903
3500175 31.54589557647705
3600180 32.43953275680542
3700185 33.33024525642395
3

30301515 276.3225815296173
30401520 277.2324984073639
30501525 278.1497528553009
30601530 279.0662055015564
30701535 279.9833381175995
30801540 280.9035665988922
30901545 281.8225166797638
31001550 282.7413582801819
31101555 283.66345834732056
31201560 284.57537817955017
31301565 285.49353551864624
31401570 286.4119303226471
31501575 287.32899594306946
31601580 288.2468795776367
31701585 289.16980481147766
31801590 290.08907079696655
31901595 291.01279759407043
32001600 291.92983388900757
32101605 292.8561725616455
32201610 293.78232979774475
32301615 294.7030804157257
32401620 295.6263518333435
32501625 296.54749870300293
32601630 297.4648027420044
32701635 298.37340807914734
32801640 299.3034334182739
32901645 300.2278108596802
33001650 301.14843130111694
33101655 302.07398295402527
33201660 302.9926793575287
33301665 303.92354679107666
33401670 304.85318088531494
33501675 305.7733542919159
33601680 306.70328664779663
33701685 307.6294832229614
33801690 308.5399765968323
33901695 309

In [8]:
#len(en_list)

21223864

In [4]:
#from nltk.tokenize import WordPunctTokenizer
#tokenizer = WordPunctTokenizer()

In [5]:
#with open('/srv/hd5/data/vprov/Poems/data/OpenSubtitlesv2018/train.en', 'w') as f:
#    for sent in en_list:
#        f.write(' '.join(tokenizer.tokenize(sent)) + '\n')
#        
#with open('/srv/hd5/data/vprov/Poems/data/OpenSubtitlesv2018/train.ru', 'w') as f:
#    for sent in ru_list:
#        f.write(' '.join(tokenizer.tokenize(sent)) + '\n')

NameError: name 'en_list' is not defined

In [65]:
#from preprocessing.text_preprocessing import create_train_bpe

In [2]:
#create_train_bpe('/srv/hd5/data/vprov/Poems/data/OpenSubtitlesv2018/')

Learning BPE...
Writing train files...
Learning BPE...
Writing train files...


## START HERE

In [2]:
import tensorflow as tf
import keras.layers as L
import numpy as np
from models.utils import infer_length, infer_mask
from models.utils import select_values_over_last_axis, compute_logits, compute_loss, compute_bleu
from models.attention_model import BasicModel, AttentionLayer

Using TensorFlow backend.


In [3]:
tf.__version__

'1.10.1'

In [4]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [5]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7051195896135322106
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11273211085
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12510337977484714237
physical_device_desc: "device: 0, name: Tesla K40m, pci bus id: 0000:03:00.0, compute capability: 3.5"
]


In [6]:
TRAIN_DIR = '/srv/hd5/data/vprov/Poems/data/OpenSubtitlesv2018/'

In [7]:
firstn=10000000

In [8]:
en_lines = []
ru_lines = []
with open(TRAIN_DIR + 'train.bpe.ru', 'r') as f:
    for i, line in enumerate(f):
        ru_lines.append(line)
        if i > firstn:
            break
        
with open(TRAIN_DIR + 'train.bpe.en', 'r') as f:
    for i, line in enumerate(f):
        en_lines.append(line)
        if i > firstn:
            break        

In [9]:
len(en_lines)

10000002

In [10]:
#data_out = np.array(ru_lines[:firstn])
#data_inp = np.array(en_lines[:firstn])

In [11]:
#data_inp.shape, data_out.shape

In [12]:
import gc
gc.collect()

0

In [13]:
from sklearn.model_selection import train_test_split
train_inp, dev_inp, train_out, dev_out = train_test_split(en_lines, ru_lines, test_size=10000,
                                                          random_state=42)
for i in range(3):
    print('inp:', train_inp[i])
    print('out:', train_out[i], end='\n\n')

inp: Looks like they did . DA@@ PH@@ N@@ E :

out: Похоже , им это удалось .


inp: Bill Carson !

out: Билл Кар@@ сон !


inp: She knows every corner of that room .

out: Она знает каждый угол в той комнате .




In [14]:
len(train_inp)

9990002

In [15]:
from models.utils import Vocab
inp_voc = Vocab.from_lines(train_inp)
out_voc = Vocab.from_lines(train_out)

In [16]:
class SpearTranslate(BasicModel):
    def __init__(self, name, inp_voc, out_voc, config,
                 emb_size=64, hid_size=128, attn_size=128):
        """ Translation model that uses attention. See instructions above. """
        self.name = name
        self.inp_voc = inp_voc
        self.out_voc = out_voc
        self.hid_size=hid_size

        with tf.variable_scope(name):
            
            # define model layers
            self.emb_inp = L.Embedding(len(inp_voc), emb_size)
            self.emb_out = L.Embedding(len(out_voc), emb_size)
            with tf.variable_scope("forward"):
                self.enc0 = tf.nn.rnn_cell.LSTMCell(hid_size, forget_bias=1.0, state_is_tuple=False)
            with tf.variable_scope("backward"):
                self.enc1 = tf.nn.rnn_cell.LSTMCell(hid_size, forget_bias=1.0, state_is_tuple=False)
                
            self.attention = AttentionLayer('Attention', hid_size *4, hid_size, hid_size)
            self.dec_start = L.Dense(hid_size*2)
            
            self.dec0 = tf.nn.rnn_cell.LSTMCell(hid_size, forget_bias=1.0, state_is_tuple=False)
            self.logits = L.Dense(len(out_voc))
            
            # prepare to translate_lines
            self.inp = tf.placeholder('int32', [None, None])
            self.initial_state = self.prev_state = self.encode(self.inp)
            self.prev_tokens = tf.placeholder('int32', [None])
            self.next_state, self.next_logits = self.decode(self.prev_state, self.prev_tokens)

        self.weights = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=name)

    def encode(self, inp, **flags):
        """
        Takes symbolic input sequence, computes initial state
        :param inp: matrix of input tokens [batch, time]
        :return: a list of initial decoder state tensors
        """
        
        # encode input sequence, create initial decoder states
        inp_lengths = infer_length(inp, self.inp_voc.eos_ix)
        mask = tf.expand_dims(tf.cast(infer_mask(inp, self.inp_voc.eos_ix), tf.bool), axis=2)

        inp_mask = tf.tile(mask, (1, 1, self.hid_size))
        #inp_mask = tf.cast(infer_mask(inp, self.inp_voc.eos_ix), tf.bool)
        inp_emb = self.emb_inp(inp)
        
        with tf.variable_scope('enc0'):
            enc_outputs_t, enc_last_t = tf.nn.bidirectional_dynamic_rnn(
                              self.enc0, self.enc1, inp_emb,
                              sequence_length=inp_lengths,
                              dtype=inp_emb.dtype)
            #[batch, time, hid_size*4]
            enc_outputs = tf.concat(enc_outputs_t, 2)
            #[batch, hid_size * 4]
            enc_last = tf.concat(enc_last_t, 1)
            
        dec_start = self.dec_start(enc_last)
        
        # apply attention layer from initial decoder hidden state
        attn, first_attn_probas = self.attention(enc_outputs, dec_start, inp_mask)
        
        # Build first state: include
        # * initial states for decoder recurrent layers
        # * encoder sequence and encoder attn mask (for attention)
        # * make sure that last state item is attention probabilities tensor
        
        first_state = [dec_start, enc_outputs, inp_mask, first_attn_probas]
        return first_state

    def decode(self, prev_state, prev_tokens, **flags):
        """
        Takes previous decoder state and tokens, returns new state and logits
        :param prev_state: a list of previous decoder state tensors
        :param prev_tokens: previous output tokens, an int vector of [batch_size]
        :return: a list of next decoder state tensors, a tensor of logits [batch,n_tokens]
        """
        # Unpack your state: you will get tensors in the same order that you've packed in encode
        [prev_dec, enc_outputs, inp_mask, first_attn_probas] = prev_state
        
        
        # Perform decoder step
        # * predict next attn response and probas given previous decoder state
        # * use prev tokens and next attn response to update decoder states
        # * predict logits
        prev_emb = self.emb_out(prev_tokens[:,None])[:,0]
        
        next_attn_response, next_attn_probas = self.attention(enc_outputs, prev_dec, inp_mask)
        
        with tf.variable_scope('dec0'):
            dec_input = tf.concat([prev_emb, next_attn_response], axis=-1)
            new_dec_out, new_dec_state = self.dec0(dec_input, prev_dec)
        
        output_logits = self.logits(new_dec_out)
        
        # Pack new state:
        # * replace previous decoder state with next one
        # * copy encoder sequence and mask from prev_state
        # * append new attention probas
        
        next_state = [new_dec_state, enc_outputs, inp_mask, next_attn_probas]
        return next_state, output_logits

In [17]:
class Config:
    pass

In [18]:
config = Config()

In [19]:
#config.enc_dim=64
#config.dec_dim=64
#config.dec_layer_size = 3

In [20]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [21]:
model = SpearTranslate('Spear', inp_voc, out_voc, config, emb_size=100, hid_size=256, attn_size=256)

Instructions for updating:
seq_dim is deprecated, use seq_axis instead
Instructions for updating:
batch_dim is deprecated, use batch_axis instead


In [22]:
inp = tf.placeholder('int32', [None, None])
out = tf.placeholder('int32', [None, None])

loss = compute_loss(model, inp, out, out_voc)
train_step = tf.train.AdamOptimizer().minimize(loss)

(?, ?)
(?, ?, 10701)


In [23]:
tf.summary.scalar("train_loss", loss)

<tf.Tensor 'train_loss:0' shape=() dtype=string>

In [24]:
metrics = {'train_loss': [], 'dev_bleu': []}
sess.run(tf.global_variables_initializer())

In [25]:
len(train_inp)

9990002

In [26]:
from tqdm import tqdm_notebook, trange
from IPython.display import clear_output
import matplotlib.pyplot as plt
import pickle

In [27]:
EPOCH = 5
INP_SIZE = len(train_inp)

In [41]:
batch_size = 64

In [29]:
saver = tf.train.Saver()

In [30]:
train_writer = tf.summary.FileWriter('./tmp/tensorboard/')

In [31]:
merged = tf.summary.merge_all()

In [32]:
summary_n = tf.Summary()

In [39]:
from tqdm import tqdm_notebook

In [ ]:
try:
    for i in range(EPOCH):
        #while cur_ix < INP_SIZE:
        cur_ix = 0
        for _ in tqdm_notebook(range(INP_SIZE//batch_size)):
            step = i * INP_SIZE + cur_ix
            feed_dict = {
                inp: inp_voc.to_matrix(train_inp[cur_ix: cur_ix+batch_size]),
                out: out_voc.to_matrix(train_out[cur_ix: cur_ix+batch_size]),
            }
            cur_ix += batch_size
            
            loss_t, _, summary = sess.run([loss, train_step, merged], feed_dict)
            train_writer.add_summary(summary, step)
            metrics['train_loss'].append((step, loss_t))
            if step % 500 == 5:
                bleus = []
                cur_dev_ix = 0
                dev_len = len(dev_inp)
                while cur_dev_ix < dev_len:
                    bleus.append(compute_bleu(model, 
                                              dev_inp[cur_dev_ix:cur_dev_ix+batch_size], 
                                              dev_out[cur_dev_ix:cur_dev_ix+batch_size], 
                                              sess, inp_voc, out_voc))
                    cur_dev_ix += batch_size
                
                metrics['dev_bleu'].append((step, np.mean(bleus)))
               
                clear_output(True)
                plt.figure(figsize=(12,4))
                for i, (name, history) in enumerate(sorted(metrics.items())):
                    plt.subplot(1, len(metrics), i + 1)
                    plt.title(name)
                    plt.plot(*zip(*history))
                    plt.grid()
                plt.show()
                print("Mean loss=%.3f" % np.mean(metrics['train_loss'][-100:], axis=0)[1], flush=True)
                print("EPOCH: ", i)
                # Tensorboard
                summary_n.value.add(tag=tagname, simple_value=metrics['dev_bleu'][-1])
                train_writer.add_summary(summary_n, step)
                train_writer.flush()
            if step % 1000 == 0:
                saver.save(sess, "/tmp/model1.ckpt")
                with open('/tmp/metrics.pkl', 'wb') as f:
                    pickle.dump(metrics, f)
except Exception as e:
    with open('errlog.txt', 'w') as f:
        f.write(str(e))
    